<a href="https://colab.research.google.com/github/E-xxi/kaggle_house-price/blob/main/houseprice_categrical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# House Prices - Advanced Regression Techniques
sale prices를 예측

### 설정

In [43]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

%cd /content/gdrive/My Drive/Kaggle
%pwd


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Kaggle


'/content/gdrive/My Drive/Kaggle'

In [40]:
# 데이터 다운받기
!kaggle competitions download -c house-prices-advanced-regression-techniques
!ls

%mkdir input/house-price
%mv house-prices-advanced-regression-techniques.zip input/house-price
%cd input/house-price/

!unzip \*.zip  && rm *.zip

  0% 0.00/199k [00:00<?, ?B/s]
100% 199k/199k [00:00<00:00, 21.8MB/s]
house-prices-advanced-regression-techniques.zip  kaggle.json
input						 lecture07_Project02_2020113677
mkdir: cannot create directory ‘input/house-price’: File exists
/content/gdrive/My Drive/Kaggle/input/house-price
Archive:  house-prices-advanced-regression-techniques.zip
replace data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### 연결

In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/content/gdrive/My Drive/Kaggle/input/house-price'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%pwd
%cd input/house-price/

data = pd.read_csv('train.csv')
OriginData = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

/content/gdrive/My Drive/Kaggle/input/house-price/test.csv
/content/gdrive/My Drive/Kaggle/input/house-price/train.csv
/content/gdrive/My Drive/Kaggle/input/house-price/data_description.txt
/content/gdrive/My Drive/Kaggle/input/house-price/sample_submission.csv
/content/gdrive/My Drive/Kaggle/input/house-price/result1.csv
/content/gdrive/My Drive/Kaggle/input/house-price/result2.csv
/content/gdrive/My Drive/Kaggle/input/house-price/result3.csv
/content/gdrive/My Drive/Kaggle/input/house-price


# Control categorical value

data description에서 ordinal variable를 고른다.


---

## 1. Ordinal Encoding

Ordinal variables

- Utilities, ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, HeatingQC, CentralAir(Y/N), KitchenQual, FireplaceQu, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC


In [45]:
#feature와 target 설정
from sklearn.model_selection import train_test_split
#train할수없는 SalePrice가 NaN인 경우 행삭제
data.dropna(axis=0, subset=['SalePrice'], inplace = True)

y = data.SalePrice
data.drop(['SalePrice'],axis=1, inplace = True)

X = data.select_dtypes(exclude=['object'])

In [ ]:
ordinal_var= ['Utilities', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'KitchenQual',
              'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC' ] 

cate_X = pd.concat([X,data[ordinal_var]],axis = 1)

cate_X.head()

numerical 및 선택한 일부 ordinal variable를 함께 가지는 cate_X를 생성해줬다.

In [65]:
#우선적으로 결측값 처리를 한다. 
threshold = 0.7

#Dropping columns with missing value rate higher than threshold
cate_X = cate_X[cate_X.columns[cate_X.isnull().mean() < threshold]]
#Dropping rows with missing value rate higher than threshold
cate_X = cate_X.loc[cate_X.isnull().mean(axis=1) < threshold]

In [66]:
X_train, X_valid, y_train, y_valid = train_test_split(cate_X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)
print(X_train.shape)

(1168, 53)


결측값 처리로 일부 열이 사라졌음으로 새로 column 리스트를 생성한다.

In [69]:
columns_list = list(cate_X.columns)
print(columns_list)

obj_X = list(cate_X.select_dtypes(['object']).columns)
print(obj_X)

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'Utilities', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'KitchenQual', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive']
['Utilities', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'KitchenQual', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive']


In [70]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
X_train[obj_X] = ordinal_encoder.fit_transform(X_train[obj_X])
X_valid[obj_X] = ordinal_encoder.transform(X_valid[obj_X])



---
## 2. control missing values : 0.15209


결측값 처리가 필요한 열을 파악한다. 

In [71]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [73]:
# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage     212
MasVnrArea        6
GarageYrBlt      58
BsmtQual         28
BsmtCond         28
BsmtExposure     28
BsmtFinType1     28
BsmtFinType2     29
FireplaceQu     551
GarageFinish     58
GarageQual       58
GarageCond       58
dtype: int64


LotFrontage, MasVnrArea, GarageYrBlt에서 결측값을 확인할 수 있다. 

- LotFrontage, MasVnrArea, GarageYrBlt:  관련한 다른 feature들에는 값이 존재하므로 extansion이 필요하진 않다. 


---



### imputation을 이용하여 결측값 처리







In [74]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [75]:
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

18730.180479452054


처리한 데이터 전체를 넣어서 모델을 확인한다.

In [81]:
from sklearn.ensemble import RandomForestRegressor
estimat_num = [10,100,200,500,1000]
for e in estimat_num:
  choosing_final_model = RandomForestRegressor(n_estimators = e, min_samples_split = 20)
  choosing_final_model.fit(imputed_X_train, y_train)
  preds = choosing_final_model.predict(imputed_X_valid)
  print(mean_absolute_error(y_valid, preds))

20004.54866532961
18132.09587298807
18349.15315375354
18216.855683083926
18204.845206037033


In [ ]:
best_esti = 100

In [83]:
split_num = [10,50, 100,200,500]
for e in estimat_num:
  choosing_final_model = RandomForestRegressor(n_estimators = best_esti, min_samples_split = e)
  choosing_final_model.fit(imputed_X_train, y_train)
  preds = choosing_final_model.predict(imputed_X_valid)
  print(mean_absolute_error(y_valid, preds))

17837.675817804826
23317.62703508772
28288.973595728614
35392.508382740714
57066.360047616814


In [84]:
best_split = 10

위의 비교로 설정한 값을 이용해 최종 randomforestregressor 모델을 확정한다. 

In [107]:
imputer = SimpleImputer()
cate_X[obj_X] = ordinal_encoder.fit_transform(cate_X[obj_X])
imputed_X = pd.DataFrame(imputer.fit_transform(cate_X))

final_model = RandomForestRegressor(n_estimators = best_esti, min_samples_split = best_split)
final_model.fit(imputed_X, y)

RandomForestRegressor(min_samples_split=10)

In [109]:
X_test = test[columns_list]
X_test[obj_X] = ordinal_encoder.fit_transform(X_test[obj_X])
imputed_X_test = pd.DataFrame(imputer.fit_transform(X_test))

# Generate test predictions
preds_test = final_model.predict(imputed_X_test)

# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.Id,
                       'SalePrice': preds_test})
output.to_csv('result4.csv', index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
